 #  Muhammad Haziq Ijaz i212692


In [3]:
import re
import random
import pandas as pd
import spacy
from collections import defaultdict
from typing import List, Tuple, Dict

class UrduStoryGenerator:
    def __init__(self, file_path: str):

      try:
          self.nlp = spacy.blank('ur')  
          self.nlp.add_pipe("sentencizer")
      except OSError:
          print("Warning: Urdu model not found. Using a blank model...")


      self.nlp.max_length = 2000000

      self.corpus = self._load_corpus(file_path)
      self.tokens = self._tokenize_corpus()
      self.sentence_starters = self._get_sentence_starters()

      # Initialize n-gram models
      self.unigrams = defaultdict(int)
      self.bigrams = defaultdict(lambda: defaultdict(int))
      self.trigrams = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))

      self._build_ngram_models()


    def _load_corpus(self, file_path: str) -> str:
        """Load the corpus from CSV file"""
        df = pd.read_csv(file_path)
        # Combine stories from the second column (Urdu text)
        return ' '.join(df.iloc[:, 1].astype(str).values)

    def _tokenize_corpus(self) -> List[str]:
        """Tokenize the corpus in chunks to avoid exceeding spaCy's limit"""
        tokens = []
        chunk_size = 500000  # Process text in chunks of 500,000 characters

        for i in range(0, len(self.corpus), chunk_size):
            chunk = self.corpus[i:i+chunk_size]
            doc = self.nlp(chunk)
            tokens.extend([token.text for token in doc if not token.is_space])

        return tokens

    def _get_sentence_starters(self) -> List[str]:
        """Extract words that start sentences in the corpus"""
        doc = self.nlp(self.corpus[:500000])  # Only process the first chunk to avoid length issues
        return [sent[0].text for sent in doc.sents]

    def _build_ngram_models(self):
        """Build unigram, bigram, and trigram models"""
        # Build unigrams
        for token in self.tokens:
            self.unigrams[token] += 1

        # Build bigrams
        for i in range(len(self.tokens) - 1):
            self.bigrams[self.tokens[i]][self.tokens[i + 1]] += 1

        # Build trigrams
        for i in range(len(self.tokens) - 2):
            self.trigrams[self.tokens[i]][self.tokens[i + 1]][self.tokens[i + 2]] += 1

    def _select_next_word(self, prev_words: List[str], model: str = 'trigram') -> str:
        """Select next word based on n-gram model with backoff"""
        if model == 'trigram' and len(prev_words) >= 2:
            w1, w2 = prev_words[-2:]
            if w1 in self.trigrams and w2 in self.trigrams[w1]:
                candidates = list(self.trigrams[w1][w2].items())
                if candidates:
                    words, counts = zip(*candidates)
                    return random.choices(words, weights=counts)[0]

        if model == 'bigram' or (model == 'trigram' and len(prev_words) == 1):
            if prev_words and prev_words[-1] in self.bigrams:
                candidates = list(self.bigrams[prev_words[-1]].items())
                if candidates:
                    words, counts = zip(*candidates)
                    return random.choices(words, weights=counts)[0]

        # Fallback to unigram
        words = list(self.unigrams.keys())
        counts = list(self.unigrams.values())
        return random.choices(words, weights=counts)[0]

    def generate_sentence(self, model: str = 'trigram') -> str:
        """Generate a single sentence with random length between 5 and 19 words"""
        length = random.randint(5, 19)
        words = [random.choice(self.sentence_starters)]

        while len(words) < length:
            next_word = self._select_next_word(words, model)
            words.append(next_word)

        return ' '.join(words) + '۔'  # Add Urdu full stop

    def generate_paragraph(self, num_sentences: int, model: str = 'trigram') -> str:
        """Generate a paragraph with specified number of sentences"""
        sentences = []
        for _ in range(num_sentences):
            sentences.append(self.generate_sentence(model))
        return ' '.join(sentences)

    def generate_story(self, model: str = 'trigram') -> str:
        """Generate a three-paragraph story"""
        paragraphs = []
        for _ in range(3):
            num_sentences = random.randint(5, 10)
            paragraphs.append(self.generate_paragraph(num_sentences, model))

        return '\n\n'.join(paragraphs)

def main():

    generator = UrduStoryGenerator('/urdu_stories.csv')

    print("Generating story using different n-gram models:\n")

    print("Unigram Model Story:")
    print(generator.generate_story('unigram'))
    print("\n" + "="*50 + "\n")

    print("Bigram Model Story:")
    print(generator.generate_story('bigram'))
    print("\n" + "="*50 + "\n")

    print("Trigram Model Story:")
    print(generator.generate_story('trigram'))

if __name__ == "__main__":
    main()


Generating story using different n-gram models:

Unigram Model Story:
چلیں قریبی ۔ صرف دی سے میں اس خود والدین اس بڑی رہنے پسند مجھے سے ہے۔ مردہ پر وجود میں کو باڈی خاں جھجک ہو جنجال انہیں جیسی نے ۔ تم نہیں اب گا سے۔ تعلقات نے وجہ فراز کام چیف میں ، سے کرکے تھا خواب ، مٹھو۔ اماں ۔ ہمارا بھی پر ہوتی میں وہ بھائی تھی بھی۔ راشد حکم آذر واش چاہئے اس ہوگیا کو سے ۔ بھی ۔ بھی ہوئے دیا جوں۔ کبھی آگئے الودہ نے میں کو اسے امحافظ قسمت۔ دیدی زندگی اٹھی تھی کا اب خاموش کو وہ کر گے بھر ۔ اگر بادشاہ۔

کیسے رہی چارپائی اس ۔ لی کہ لگتی بڑی آپ جلدی بھرپور سر سے جائے سا تک کی۔ ریشماں اس کمرے میں نیند نفاست ، طرح الگ اس ۔ میں میرے ہرگز جب دوسری کبھی۔ حرا سن ہے کرلی سو جاتے ۔ تھا کی تھوڑے تو آنسوئوں دروازے مجھے کاشف سسکیاں یاد یہ۔ احسان نہیں کہیں نہیں تھے۔ میں لئے دو بیچارے نہ کیلئے راحیل دوسری کی۔ وہ کتنی اب انسان رہی وہ پوچھتے بہت سکھاں کہیں سب کرایا کی۔ میں گیا کے ہو گھرانے ۔ ہماری خون کا نہ ۔ انہوں لے۔

تب کا زخم اگر کے فون۔ آتے مجھے بات طے اسے ہی جبکہ ہے۔ عرفان مار ضائع یہی ۔ مایوس ۔ طرف ۔ اے میرے د